# <span style="color:#54B1FF">Exploring Data:</span> &nbsp; <span style="color:#1B3EA9"><b>Write Results</b></span>

<br>



<br>

⚠️ **NOTE!**  &nbsp; &nbsp; All data files are saved in the same directory as this notebook.

<br>


___

<a name="parse-write"></a>
## Writing results
[Back to Table of Contents](#toc)
<br>

The `names` variable from the previous section is a list of strings, but the `prices` variable is a list of integers. One way to save these data easily is to use `csv.writer`:

In [24]:
dir0     = os.path.abspath('')       # directory in which this notebook is saved
fnameCSV = os.path.join( dir0, 'kakaku-prices.csv' )

with open(fnameCSV, 'w') as fid:     # open in write mode
    writer = csv.writer(fid)         # create a writer object
    labels = ['Num','Name','Price']  # column labels
    writer.writerow( labels )        # write column labels
    for i,(n,p) in enumerate(zip(names,prices)):   # cycle through rows
        writer.writerow( [i+1,n,p] ) # write the current row to file

If the cell above executed without errors, then there should be a new file called **kakaku-prices.csv** in the same folder as this workbook. Check that it contains titles and prices for 200 books.

Last, let's assemble all of our code into a single script (i.e., a single cell in Jupyter that can be run by itself).